<a href="https://colab.research.google.com/github/ShaliniR8/lahman-mlb/blob/main/combined_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn import preprocessing
from sklearn import compose
from sklearn import impute
from sklearn import decomposition
from sklearn import model_selection
from sklearn.utils import compute_class_weight
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics

In [ ]:
%%capture
!pip install xgboost

In [ ]:
#@title Load Dataset

%%capture
# ---- filtered (only from 1910) 
# !wget https://github.com/ShaliniR8/lahman-mlb/raw/main/baseballdatabase.zip
# !unzip /content/baseballdatabase.zip -d "/content/"


#---- full
!wget https://github.com/ShaliniR8/lahman-mlb/raw/main/baseballdatabank_2022.zip
!unzip /content/baseballdatabank_2022.zip -d "/content/"

path = "/content/baseballdatabank-2022.2/core/"



In [ ]:
batting = pd.read_csv(path + "Batting.csv")
pitching = pd.read_csv(path + "Pitching.csv")
people = pd.read_csv(path + "People.csv")
# fielding = pd.read_csv(path + "Fielding.csv")
teams = pd.read_csv(path + "Teams.csv")

In [ ]:
y = teams['WSWin']
batting = batting[["playerID","yearID","teamID", "H", "BB", "HBP", "AB", "SF", "2B", "3B", "HR"]].copy()
pitching = pitching[["playerID","yearID","teamID", "ERA"]].copy()
people = people[['playerID', 'birthYear']].copy()
teams = teams[['teamID', 'yearID', 'W', 'L']].copy()
all_dfs = { 
    'batting': batting,
    'pitching': pitching,
    'people':people,
    # 'fielding':fielding,
    'teams':teams
}

## Cleaning

In [ ]:
from re import M
def cleanX(X):
  categorical_features = X.select_dtypes("object").columns.to_list()
  numerical_features = X.select_dtypes(["int64", "float64"]).columns.to_list()
  knnImp = impute.KNNImputer(missing_values=np.nan)
  simpleImp = impute.SimpleImputer(strategy = 'median', missing_values=np.nan)
  Ximputer = compose.ColumnTransformer([
    ('categorical', knnImp, categorical_features),
    ('numerical', simpleImp, numerical_features)
  ])
  X_ = Ximputer.fit_transform(X)

  return X_

In [ ]:
for name,df in all_dfs.items():
  not_keys = list(set(df.columns.to_list()) - {'teamID', 'playerID'})
  keys = list(set(df.columns.to_list()) - set(not_keys))
  imputedX = pd.DataFrame(cleanX(df[not_keys]), columns = not_keys, dtype='int64')
  all_dfs[name] = pd.concat([df[keys], imputedX], axis = 1)
  print(name)
  print(all_dfs[name].info())
  


batting
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110495 entries, 0 to 110494
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   playerID  110495 non-null  object
 1   teamID    110495 non-null  object
 2   3B        110495 non-null  int64 
 3   SF        110495 non-null  int64 
 4   H         110495 non-null  int64 
 5   BB        110495 non-null  int64 
 6   AB        110495 non-null  int64 
 7   HR        110495 non-null  int64 
 8   2B        110495 non-null  int64 
 9   yearID    110495 non-null  int64 
 10  HBP       110495 non-null  int64 
dtypes: int64(9), object(2)
memory usage: 9.3+ MB
None
pitching
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49430 entries, 0 to 49429
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   playerID  49430 non-null  object
 1   teamID    49430 non-null  object
 2   yearID    49430 non-null  int64 
 3   ERA    

## Constructing Features

In [ ]:
def win_perc(team, year):
    team_year = teams[(teams['yearID'] == year) & (teams['teamID'] == team)]
    if team_year.empty:
        return np.nan
    team_year = team_year.iloc[0]
    w = team_year['W']
    l = team_year['L']
    return w / (w + l)


def get_rolling_win_perc (row):
    year = row['yearID']
    team = row['teamID']
    year1 = win_perc(team, year - 1)
    year2 = win_perc(team, year - 2)
    year3 = win_perc(team, year - 3)
    year4 = win_perc(team, year - 4)
    year5 = win_perc(team, year - 5)
    if (year1 == np.nan or year2 == np.nan or year3 == np.nan or year3 == np.nan or year4 == np.nan or year5 == np.nan):
      if (year1 == np.nan or year2 == np.nan or year3 ==np.nan):
        if year1 == np.nan: 
          return (year, team, np.nan, np.nan, np.nan)
        else:
          return (year, team, year1, np.nan, np.nan)
      else:
        return (year, team, year1, (year1 + year2 + year3)/3, np.nan )
    else: 
        return (year, team, year1, (year1 + year2 + year3)/3, (year1 + year2 + year3 + year4 + year5)/5)


In [ ]:

def get_age(row):
  playerID = row['playerID']
  yearID = row['yearID']
  birthYear = people[people['playerID'] == playerID]['birthYear']
  birthYear = birthYear.iloc[0]
  age = yearID - birthYear
  row['age'] = age
  return row

In [ ]:
def ops_calc(row):
  #calc done on rows of batting table
  if row['AB'] == 0:
      return np.nan
  HBP = row["HBP"]
  SF = row["SF"]
  singles = row["H"] - row["2B"] - row["3B"] - row["HR"]
  obp = (row["H"] + row["BB"] + HBP) / (row["AB"] + row["BB"] + SF + HBP)
  slug = (singles + (2 * row["2B"]) + (3 * row["3B"]) + (4 * row["HR"])) / row["AB"]
 
  return slug + obp 

def get_ops(row):
  year = row['yearID'] - 1
  team = row['teamID']
  player = row["playerID"]
  row['ops'] = np.nan
  filtered_df = batting[(batting['yearID'] == year) & (batting['teamID'] == team) & (batting['playerID'] == player)]
  if filtered_df.empty:
      return row
  filtered_df = filtered_df.iloc[0]
  row['ops'] = ops_calc(filtered_df)
  return row

In [ ]:
teams = all_dfs['teams']
pitching = all_dfs['pitching']
people = all_dfs['people']
batting = all_dfs['batting']

In [ ]:
# teams winning perc
p = teams.apply(lambda x: get_rolling_win_perc(x), axis = 1).copy()
t_features = pd.DataFrame(list(p), columns = ['yearID', 'teamID', 'prev_year_win_perc', 'rolling_3_win_perc', 'rolling_5_win_perc'])

In [ ]:
#pitcher ages
ages = pitching.apply(lambda x: get_age(x), axis = 1).copy()

In [ ]:
#median era and mean pitching age
p_features = ages.groupby(['teamID', 'yearID']).agg({'ERA':'median', 
                                                            'age':'mean'}).reset_index()
p_features = p_features.rename(columns = {"ERA": "medianERA", 'age': 'pitcherAverageAge'})

In [ ]:
#batter ages
ages = batting.apply(lambda x: get_age(x), axis = 1).copy()

In [ ]:
ops = ages.apply(lambda x: get_ops(x), axis=1).copy()

In [ ]:
#median ops and average age

b_features = ops.groupby(['teamID', 'yearID']).agg({'age':'mean', 
                                                            'ops':'median'}).reset_index()
b_features = b_features.rename(columns = {"age": "batterAverageAge", "ops": "medianOPS"})

